# Fuelio Fuel Consumption Analysis

This notebook provides comprehensive analysis of fuel consumption data exported from the Fuelio mobile app. It processes odometer readings, fuel volumes, and pricing data to generate insights into vehicle usage patterns and fuel economy trends over time.

**Key Features:**
- Time-series analysis of odometer readings
- Monthly distance traveled metrics and aggregated statistics
- Fuel consumption and pricing trend analysis
- Statistical visualizations (histograms, boxplots, violin plots)

**Data Source:** CSV export from Fuelio app, containing fields such as Date, Odo (km), Fuel (L), and Price.

## Library Imports

Import essential Python libraries required for data analysis:
- **pandas**: Data manipulation and analysis
- **numpy**: Numerical computing and array operations
- **matplotlib**: Plotting and visualization
- **seaborn**: Statistical data visualization
- **os**: File path and environment variable handling
- **calendar**: Month name conversions for time-based grouping
- **dataframe_image**: Export DataFrames as images for reports and documentation

In [ ]:
# Import required libraries for data analysis and visualization
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import dataframe_image as dfi

## Data Loading and Preparation

This section determines the appropriate Fuelio CSV file to load and prepares it for analysis:

**Data Source Priority:**
1. Custom file specified via `FUELIO_CSV_PATH` environment variable
2. Latest CSV from `CarAnalysis_database` submodule (if available)
3. Default `Fuelio_sample.csv` file for demonstration purposes

**Preprocessing Steps:**
- Skip header rows (first 4 lines) and footer metadata from Fuelio export
- Dynamically count footer lines by detecting the CostCategories marker
- Sort entries by odometer reading to ensure chronological order
- Convert Date column to datetime format for time-series analysis

This prepares the dataset for subsequent visualizations and metrics computed in later cells.

In [ ]:
csv_path = os.getenv('FUELIO_CSV_PATH')
if not csv_path:
    # Check if CarAnalysis_database submodule has data
    if os.path.exists('CarAnalysis_database') and any(f.endswith('.csv') for f in os.listdir('CarAnalysis_database') if f.startswith('Fuelio')):
        csv_files = [f for f in os.listdir('CarAnalysis_database') if f.startswith('Fuelio') and f.endswith('.csv')]
        csv_path = os.path.join('CarAnalysis_database', sorted(csv_files)[-1])  # Use most recent file
    else:
        csv_path = 'Fuelio_sample.csv'

    # Count footer rows to skip (lines starting with "##" at end of CSV)
    with open(csv_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        footer_count = 1
        for line in reversed(lines):
            if  'CostCategories' not in line.strip():
                footer_count += 1
            else:
                break

print(f"Loading data from: {csv_path} (skipping {footer_count} footer lines)")

## Load and Preprocess CSV Data

This cell executes the actual data loading and preprocessing operations:

**Operations Performed:**
1. **Load CSV**: Read the determined CSV file using pandas, skipping header and footer rows
2. **Sort by Odometer**: Ensure chronological order based on odometer readings
3. **Date Conversion**: Convert Date column to datetime format for time-series operations
4. **Preview**: Display first 5 rows to verify successful loading

The resulting `fuel_data` DataFrame contains all refueling records ready for analysis.

In [ ]:
print(f'Using data from: {csv_path}')
fuel_data = pd.read_csv(
    csv_path, skiprows=[0, 1, 2, 3], skipfooter=footer_count, engine='python')
fuel_data.sort_values(by="Odo (km)", inplace=True)
fuel_data["Date"] = pd.to_datetime(fuel_data["Date"])
fuel_data.head()

## Monthly Distance Metrics Calculation

This section computes monthly distance statistics to analyze vehicle usage patterns over time.

**Methodology:**
1. **Period Extraction**: Create `Year-Month` and `Year` period columns from dates
2. **Distance Calculation**: Compute `km_diff` as the odometer difference between consecutive refueling entries
3. **Monthly Aggregation**: Aggregate kilometers traveled per month using `groupby('Year-Month')`:
   - `monthly_km`: Total kilometers traveled per month
   - `monthly_mean_km` and `Avgkm/refill`: Average distance between refills in each month
   - `Maxkm/month`: Maximum distance between consecutive refills in each month
   - `Minkm/month`: Minimum distance between consecutive refills in each month
   - `Avgkm/liter`: Fuel efficiency (km per liter) for the month
4. **Mapping**: Propagate these monthly metrics back to each row in the DataFrame

**Monthly Metrics:**
- Provides insights into usage patterns on a monthly basis
- Helps identify trends in vehicle usage over time
- Enables comparison of driving patterns across different months and years

**Prerequisites:** Assumes `fuel_data` is already sorted by odometer reading with Date and Odo (km) columns.

In [ ]:
# Calculate average, minimum, and maximum km/month metrics grouped by month
def calculate_avg_km_per_month(df):
    df = df.copy()

    # Ensure period columns exist
    df['Year-Month'] = df['Date'].dt.to_period('M')
    df['Year']       = df['Date'].dt.to_period('Y')
    df["Month"]      = df["Date"].dt.month
    df['MonthName']  = df['Month'].apply(lambda x: calendar.month_name[x])
    df["Year_name"]  = df["Date"].dt.year
    df["Month_name"] = df["Date"].dt.month_name()

    # km difference between consecutive entries
    df['km_diff']    = df['Odo (km)'].diff()

    # Sum monthly km and compute monthly aggregates
    monthly_km          = df.groupby('Year-Month')['km_diff'].sum()
    monthly_mean_km     = df.groupby('Year-Month')['km_diff'].mean()
    max_km              = df.groupby('Year-Month')['km_diff'].max()
    min_km              = df.groupby('Year-Month')['km_diff'].min()
    liter               = df.groupby('Year-Month')['Fuel (L)'].sum()
    size_km             = df.groupby('Year-Month').size()

    df['monthly_km']         = df['Year-Month'].map(monthly_km).astype(float)
    df['monthly_mean_km']    = df['Year-Month'].map(monthly_mean_km).astype(float)
    df['monthly_km_size']    = df['Year-Month'].map(size_km).astype(float)
    df['Avgkm/refill']       = (df['monthly_km'] / df['monthly_km_size']).astype(float)
    df['Avgkm/liter']        = (df['monthly_km'] / df['Year-Month'].map(liter).astype(float))
    df['Maxkm/month']        = df['Year-Month'].map(max_km).astype(float)
    df['Minkm/month']        = df['Year-Month'].map(min_km).astype(float)
    df['Eur/km']             = (df['Price'] / df['km_diff']).astype(float)
    return df

fuel_data = calculate_avg_km_per_month(fuel_data)
fuel_data.head(10)

## Odometer Trend Visualization

This plot visualizes the vehicle's odometer reading progression over time, providing insights into:
- Overall vehicle usage patterns and mileage accumulation
- Periods of high vs. low usage
- Consistency of vehicle use over the data collection period

The plot is saved to `plots/odometer_vs_date.png` for archival and CI/CD workflows.

In [ ]:
# Plot odometer reading over time to visualize vehicle usage patterns
os.makedirs('plots', exist_ok=True)

plt.figure(figsize=(12, 6))
plt.plot(fuel_data["Date"], fuel_data["Odo (km)"], marker='o', linestyle='-', markersize=3)
plt.xlabel("Date")
plt.ylabel("Odo (km)")
plt.title("Odometer Reading vs Date")
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/odometer_vs_date.png', dpi=150, bbox_inches='tight')
plt.show()

## Average Kilometers per Refill Trend

This visualization displays the average kilometers per refill over time, with shaded min/max boundaries showing the range of individual trip distances.

**Visualization Elements:**
- **Blue line**: Average kilometers per refill for each month
- **Gray shaded area**: Range between minimum and maximum single trip distances in each month
- **X-axis**: Date/time progression
- **Y-axis**: Distance in kilometers

**Insights Provided:**
- Trends in typical refueling intervals (how far between fill-ups)
- Variability in trip distances within each month
- Identification of periods with unusual driving patterns (e.g., vacations, lifestyle changes)

Output saved as `plots/avg_km_per_month.png`.

In [ ]:
# Plot average km per refill with shaded min/max range to show trip distance variation
plt.figure(figsize=(20, 6))
plt.plot(fuel_data["Date"], fuel_data["Avgkm/refill"],
         marker='o', linestyle='-', markersize=3)
plt.fill_between(fuel_data["Date"],
                 fuel_data["Minkm/month"],
                 fuel_data["Maxkm/month"],
                 color='gray', alpha=0.2,
                 label='Min-Max Range')

plt.xlabel("Date")
plt.ylabel("Avg km/refill")
plt.title("Average Kilometers per Refill vs Date")
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/avg_km_per_month.png', dpi=150, bbox_inches='tight')
plt.show()

<!-- Add descriptions -->

In [ ]:
def name_in_histogram_bins(name_variable, counts, bin_edges, patches):
    for group_idx, group_patches in enumerate(patches):
        year = int(labels[group_idx])
        df_year = data_by_year[year]

        for bin_idx, rect in enumerate(group_patches):
            height = rect.get_height()
            if height == 0:
                continue

            # Bin boundaries
            left = bin_edges[bin_idx]
            right = bin_edges[bin_idx + 1]

            # Select months falling inside this bin
            months_in_bin = df_year.loc[
                (df_year[name_variable] >= left) &
                (df_year[name_variable] < right),
                "Month_name"
            ]
            # print(year, months_in_bin)
            if months_in_bin.empty:
                continue

            # Shorten text if many months
            months_text = ",\n ".join(months_in_bin.unique())

            x = rect.get_x() + rect.get_width() / 2
            y = rect.get_y() + height / 2

            plt.text(
                x, y,
                f"{year},\n{months_text}",
                ha="center",
                va="center",
                fontsize=6
            )

## Monthly Distance Distribution by Year

This stacked histogram visualizes how monthly distance traveled is distributed across different years.

**Chart Details:**
- **Type**: Stacked histogram with 20 bins
- **Colors**: Each year represented by a different color
- **X-axis**: Average km/month values
- **Y-axis**: Frequency count (stacked by year)

**Use Cases:**
- Identify typical monthly mileage ranges for different years
- Compare usage patterns year-over-year
- Detect shifts in driving behavior over time

Output saved as `plots/monthly_distance_histogram.png`.

In [ ]:
# Create histogram showing distribution of monthly distances traveled
# Ensure Year column exists

data_by_year = {
    y: g[["monthly_mean_km", "Month_name", "km/l"]]
    for y, g in fuel_data.groupby("Year_name")
}

# Prepare data grouped by year
groups      = [g["monthly_mean_km"].values for y, g in fuel_data.groupby("Year_name")]
groups_kml  = [g["km/l"].values for y, g in fuel_data.groupby("Year_name")]
labels      = [str(y) for y, g in fuel_data.groupby("Year_name")]

# print(groups, groups_kml)
# print(labels)

plt.figure(figsize=(15, 6))

counts, bin_edges, patches = plt.hist(
    groups,
    bins=20,
    stacked=True,     # <-- THIS makes the histogram stacked
    label=labels,
    alpha=0.8
)

# print(bin_edges)

plt.xlabel("Avg km/month")
plt.ylabel("Count (stacked)")
plt.title("Stacked distribution of Avg km/month by Year")
plt.legend(title="Year")
plt.tight_layout()            

name_in_histogram_bins("monthly_mean_km", counts, bin_edges, patches)

plt.savefig('plots/monthly_distance_histogram.png',
            dpi=150, bbox_inches='tight')
plt.show()

## Fuel Efficiency Distribution by Year (km/l)

This stacked histogram visualizes fuel efficiency (kilometers per liter) distribution across different years.

**Chart Details:**
- **Type**: Stacked histogram with 20 bins
- **Colors**: Each year represented by a different color
- **X-axis**: Fuel efficiency in km/l
- **Y-axis**: Frequency count (stacked by year)
- **Data Filtering**: Excludes zero values to focus on valid fuel consumption entries

**Use Cases:**
- Compare fuel efficiency patterns across years
- Identify typical fuel economy ranges for each year
- Detect improvements or degradation in fuel efficiency over time
- Analyze impact of driving conditions or vehicle maintenance on efficiency

Output saved as `plots/km_l_histogram.png`.

In [ ]:
plt.figure(figsize=(15, 6))

counts_2, bin_edges_2, patches_2 = plt.hist(
    [g[g > 0] for g in groups_kml],
    # groups_kml,
    bins=20,
    stacked=True,
    label=labels,
    alpha=0.8
)

plt.xlabel("km/l")
plt.ylabel("Count (stacked)")
plt.title("Stacked distribution of km/l by Year (excluding 0 values)")
plt.legend(title="Year")
# plt.tight_layout()

name_in_histogram_bins("km/l", counts_2, bin_edges_2, patches_2)

plt.savefig('plots/km_l_histogram.png', dpi=150, bbox_inches='tight')
plt.show()

## Fuel Efficiency Distribution by Year (Full Tank Refills Only)

This violin plot analyzes fuel efficiency (km/l) for full tank refills, providing the most accurate efficiency measurements.

**Data Filtering:**
- **Full Tank Only**: Filters data where `Full == 1` to ensure accurate km/l calculations
- Only complete fill-ups provide reliable efficiency metrics as partial fills can skew calculations

**Visualization Features:**
- **Width**: Represents density/frequency of efficiency values at each level
- **Inner markings**: Show quartiles and median values
- **Y-axis range**: Limited to 0-20 km/l for focus on realistic efficiency ranges
- **cut=0**: Prevents density estimation beyond actual data range

**Use Cases:**
- Track year-over-year changes in fuel efficiency
- Identify typical operating efficiency ranges
- Detect anomalies or unusual efficiency patterns
- Assess impact of maintenance, driving style, or fuel quality changes

**Note:** This analysis is more reliable than including partial fills, as full tank refills enable accurate km/l calculation from odometer differences.

Output saved as `plots/fuel_violin_plot.png`.

In [ ]:
fuel_data_filltank = fuel_data[fuel_data["Full"] == 1]
plt.figure(figsize=(15, 6))
sns.violinplot(
    data=fuel_data_filltank,
    x="Year",
    y="km/l",
    cut=0,
)

plt.title("Distribution of km/l by Year")
plt.xlabel("Year")
plt.ylim(0, 20)
plt.ylabel("km/l")
plt.tight_layout()
plt.savefig('plots/fuel_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Yearly Average km/month Distribution (Violin Plot)

Violin plots combine box plots with kernel density estimation to show the full distribution shape of monthly kilometers by year.

**Visualization Features:**
- **Width**: Represents density/frequency of data points at each value
- **Inner markings**: Show quartiles and median (similar to box plots)
- **Symmetry**: Each violin is mirrored for visual clarity

**Advantages over Histograms:**
- Reveals distribution shape (unimodal, bimodal, skewed)
- Shows data density at all value ranges
- Easier year-to-year comparison

**What to Look For:**
- Central tendency (median line)
- Spread of data (violin width)
- Multiple peaks indicating different usage patterns

Output saved as `plots/avg_km_violin_plot.png`.

In [ ]:
# Create violin plot showing distribution of average km/month by year
plt.figure(figsize=(15, 6))
sns.violinplot(
    data=fuel_data,
    cut=0,
    x="Year",
    y="monthly_km"
)

plt.title("Distribution of Monthly km by Year")
plt.xlabel("Year")
plt.ylabel("Monthly km")
plt.tight_layout()
plt.savefig('plots/monthly_km_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Monthly Distance Trends by Year

This section generates individual plots for each year, showing month-by-month distance trends with all 12 months displayed.

**Data Processing:**
- Creates a complete 12-month calendar (January through December)
- Merges actual data with the full calendar, filling gaps with zeros for months without data
- Aggregates monthly metrics: average km/refill, monthly total km, and min/max ranges

**Visualization Elements:**
- **Two line plots**: 
  - Average km per refill for each month
  - Total monthly kilometers traveled
- **Gray shaded area**: Range between minimum and maximum trip distances
- **Y-axis range**: Fixed at 0-4000 km for consistent year-to-year comparison

**Use Cases:**
- Identify seasonal patterns in vehicle usage
- Compare driving behavior across different months
- Detect months with missing data (zeros)
- Track consistency of usage within each year
- Spot vacation periods or unusual usage patterns

**Output:** Separate plot files saved as `plots/avg_km_violin_plot_{year}.png` for each year in the dataset.

In [ ]:
# Create violin plot showing distribution of average km/month by year
plt.figure(figsize=(15, 6))
sns.violinplot(
    data=fuel_data[fuel_data["Eur/km"] < 0.5],
    cut=0,
    x="Year",
    y="Eur/km"
)

plt.title("Distribution of Eur/km by Year")
plt.xlabel("Year")
plt.ylabel("Eur/km")
plt.tight_layout()
plt.savefig('plots/eur_km_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
all_months = pd.DataFrame({
        'Month': range(1, 13),
        'MonthName': [calendar.month_name[i] for i in range(1, 13)]
})

for year in fuel_data["Year"].unique().tolist():
        # Filter data for the current year
        year_data = fuel_data[fuel_data["Year"] == year]
        
        # Group by month to get one representative value per month
        monthly_data = year_data.groupby('Month').agg({
                'MonthName': 'first',
                'Avgkm/refill': 'mean',
                'Maxkm/month': 'max',
                'Minkm/month': 'min',
                "monthly_km": 'mean'
        }).reset_index()
        
        # Create a complete DataFrame with all 12 months        
        # Merge to include all months (even those without data)
        monthly_data = all_months.merge(monthly_data, on=['MonthName', 'Month'], how='left',)
        # Replace NaN with 0 in the selected columns
        monthly_data[['Avgkm/refill', 'Maxkm/month', 'Minkm/month', "monthly_km"]] = (
                monthly_data[['Avgkm/refill', 'Maxkm/month', 'Minkm/month', "monthly_km"]].fillna(0)
        )
        # Create plot
        plt.figure(figsize=(15, 6))
        plt.ylim(0, 4000)
        plt.plot(monthly_data["MonthName"], monthly_data["Avgkm/refill"],
                 marker='o', linestyle='', markersize=3)
        plt.plot(monthly_data["MonthName"], monthly_data["monthly_km"],
                 marker='o', linestyle='-', markersize=3)
        plt.fill_between(range(len(monthly_data)),
                         monthly_data["Minkm/month"],
                         monthly_data["Maxkm/month"],
                         color='gray', alpha=0.2,
                         label='Min-Max Range')
        plt.legend(title="Month")
        plt.xlabel("Month")
        plt.ylabel("Avg km/month")
        plt.title(f"Distribution of Avg km/month by Month in {year}")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'plots/avg_km_violin_plot_{year}.png', dpi=150, bbox_inches='tight')
        plt.show()

## Cost Per Kilometer Distribution by Year

This violin plot analyzes the cost efficiency (Euros per kilometer) over time, providing insights into the economic aspects of vehicle operation.

**Data Filtering:**
- **Outlier removal**: Filters data where `Eur/km < 0.5` to exclude unrealistic values
- Focuses on typical operating cost ranges

**Visualization Features:**
- **Width**: Represents density/frequency of cost values at each level
- **Inner markings**: Show quartiles and median values
- **cut=0**: Prevents density estimation beyond actual data range

**Use Cases:**
- Track year-over-year changes in operating costs
- Identify periods of high or low cost efficiency
- Compare cost patterns across different years
- Assess impact of fuel prices and vehicle efficiency on overall costs
- Budget planning and cost analysis

**Interpretation:**
- Lower Eur/km values indicate better cost efficiency
- Wide distributions suggest variable cost patterns
- Shifts in median values show trends in operating costs

Output saved as `plots/eur_km_violin_plot.png`.

In [ ]:
# Create violin plot showing distribution of average km/month by year
plt.figure(figsize=(15, 6))
sns.violinplot(
    data=fuel_data[fuel_data["Eur/km"] < 0.5],
    cut=0,
    x="Year",
    y="Eur/km"
)

plt.title("Distribution of Eur/km by Year")
plt.xlabel("Year")
plt.ylabel("Eur/km")
plt.tight_layout()
plt.savefig('plots/eur_km_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Average km/month by Year (Boxplot)

Traditional box-and-whisker plot providing statistical summary of monthly distance by year.

**Box Plot Components:**
- **Box**: Interquartile range (IQR) containing the middle 50% of data
- **Line inside box**: Median value (50th percentile)
- **Whiskers**: Extend to 1.5 × IQR from the quartiles
- **Dots**: Outliers beyond the whiskers

**Statistical Insights:**
- Central tendency: Where is the median?
- Variability: How wide is the IQR?
- Symmetry: Is the distribution skewed?
- Outliers: Are there exceptional months?

**Comparison with Violin Plot:** Boxplots emphasize statistical measures over distribution shape, making them ideal for quick statistical comparison.

Output saved as `plots/avg_km_boxplot.png`.

In [ ]:
# Create boxplot showing distribution of average km/month by year
# fuel_data["Year"] = fuel_data["Date"].dt.year
fuel_data.boxplot(column="monthly_km", by="Year", figsize=(15, 6))
plt.title("Distribution of Avg km/month by Year")
plt.suptitle("")
plt.xlabel("Year")
plt.ylabel("Avg km/month")
plt.savefig('plots/avg_km_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

## Fuel Price Distribution by Year

This boxplot analyzes fuel price trends over time, helping identify economic patterns and pricing volatility.

**Analysis Objectives:**
- Track year-over-year fuel price changes
- Identify periods of price stability vs. volatility
- Detect seasonal or economic price fluctuations
- Compare price ranges across different years

**Interpretation Tips:**
- Rising medians indicate increasing fuel costs
- Wide boxes/whiskers suggest high price volatility
- Outliers may represent unusual pricing events

**Note:** Price units depend on the Fuelio data export (typically per liter or gallon).

Output saved as `plots/price_boxplot.png`.

In [ ]:
# Create boxplot showing fuel price distribution by year
# fuel_data["Year"] = fuel_data["Date"].dt.year
fuel_data.boxplot(column="Price", by="Year", figsize=(15, 6))
plt.title("Distribution of Price by Year")
plt.suptitle("")
plt.xlabel("Year")
plt.ylabel("Price")
plt.savefig('plots/price_boxplot.png', dpi=150, bbox_inches='tight')
plt.show()

# Create violin plot showing distribution of average km/month by year
plt.figure(figsize=(13, 6))
sns.violinplot(
    data=fuel_data,
    cut=0,
    x="Year",
    y="Price"
)

plt.title("Distribution of Price by Year")
plt.xlabel("Year")
plt.ylabel("Price")
plt.tight_layout()
plt.savefig('plots/avg_price_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Fuel Volume Distribution by Year

This violin plot analyzes the distribution of refueling volumes over time, providing insights into tank usage patterns.

**What This Reveals:**
- Typical refueling amounts per transaction
- Whether the tank is typically filled completely vs. partial fills
- Changes in refueling behavior over time
- Distribution patterns (e.g., bimodal if mixing full/partial fills)

**Chart Configuration:**
- Y-axis limited to 0-52 liters (typical passenger vehicle tank capacity)
- `cut=0` parameter prevents density estimation beyond data range
- Volume measured in liters (L)

**Practical Applications:**
- Identify refueling habits (wait for empty vs. top-off strategy)
- Detect tank capacity if consistently hitting a ceiling
- Spot anomalies or data entry errors

Output saved as `plots/fuel_violin_plot.png`.

In [ ]:
plt.figure(figsize=(15, 6))
sns.violinplot(
    data=fuel_data,
    x="Year",
    y="Fuel (L)",
    cut=0,
)

plt.title("Distribution of Fuel (L) by Year")
plt.xlabel("Year")
plt.ylim(0, 52)
plt.ylabel("Fuel (L)")
plt.tight_layout()
plt.savefig('plots/fuel_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Fuel Efficiency Violin Plot (Full Tank Data)

This cell regenerates the fuel efficiency (km/l) violin plot using only full tank refills, ensuring the most accurate efficiency calculations.

**Note:** This plot is identical to an earlier visualization and overwrites the same output file (`plots/fuel_violin_plot.png`). The data is filtered to include only entries where the tank was completely filled (`Full == 1`), as partial fills can produce misleading efficiency calculations.

See the earlier "Fuel Efficiency Distribution by Year (Full Tank Refills Only)" section for detailed documentation of this visualization.

In [ ]:
fuel_data_filltank = fuel_data[fuel_data["Full"] == 1]
plt.figure(figsize=(15, 6))
sns.violinplot(
    data=fuel_data_filltank,
    x="Year",
    y="km/l",
    cut=0,
)

plt.title("Distribution of km/l by Year")
plt.xlabel("Year")
plt.ylim(0, 20)
plt.ylabel("km/l")
plt.tight_layout()
plt.savefig('plots/fuel_violin_plot.png', dpi=150, bbox_inches='tight')
plt.show()

## Monthly Distance Summary Table

This pivot table provides a comprehensive tabular summary of kilometers traveled, organized by month (rows) and year (columns).

**Table Structure:**
- **Rows**: Month names (January through December)
- **Columns**: Years present in the dataset
- **Values**: Total kilometers traveled in each month-year combination
- **Bottom row**: "Average Km per Year" - mean across all months for each year
- **Right column**: "Average Km per Month" - mean across all years for each month

**Data Processing:**
- Uses `pivot_table` to aggregate `km_diff` (distance between refills) by month and year
- Reindexed to ensure all 12 months appear in calendar order
- Values rounded to 2 decimal places for readability
- Missing values preserved (not filled with zeros) to distinguish no-data from zero-travel

**Use Cases:**
- Quick reference for monthly and yearly travel totals
- Identify seasonal patterns across multiple years
- Compare specific months year-over-year
- Calculate annual and monthly averages
- Export as formatted table image for reports

**Output:** Table displayed in notebook and exported as `plots/monthly_km_diff_table.png` for documentation.

In [ ]:
# Pivot tables per year: total km and fuel by month (Jan–Dec order)
month_pivot_by_year = (
    fuel_data.pivot_table(
        index="MonthName",
        columns="Year",
        values={"km_diff": "sum"},
        aggfunc="sum",
    )
    .reindex(all_months["MonthName"])
    # .fillna(0)
)
month_pivot_by_year.loc["Mean[km]"] = month_pivot_by_year.astype(float).mean(axis=0).values
month_pivot_by_year["Mean [km]"] = month_pivot_by_year.astype(float).mean(axis=1).values
month_pivot_by_year = month_pivot_by_year.round(2)
dfi.export(month_pivot_by_year, 'plots/monthly_km_diff_table.png', table_conversion='matplotlib',crop_top=False)
month_pivot_by_year

## Monthly Fuel Cost Summary Table

This pivot table summarizes total fuel costs organized by month (rows) and year (columns), providing economic insights into refueling expenses.

**Table Structure:**
- **Rows**: Month names (January through December)
- **Columns**: Years present in the dataset
- **Values**: Total fuel costs (sum of all refill prices) for each month-year combination
- **Bottom row**: "Mean [EUR]" - average monthly cost for each year
- **Right column**: "Mean [EUR]" - average cost for each month across all years

**Data Processing:**
- Aggregates `Price` field (fuel cost per transaction) by month and year
- Reindexed to maintain calendar month order (January-December)
- Rounded to 2 decimal places for currency precision
- Missing values preserved to distinguish between no refueling and zero cost

**Use Cases:**
- Track total fuel spending by month and year
- Identify seasonal cost variations (e.g., winter vs. summer driving)
- Compare year-over-year fuel expenses
- Budget planning and expense analysis
- Correlate costs with distance traveled from the km_diff table

**Note:** Currency shown as EUR (Euros), but actual currency depends on the Fuelio data export settings.

**Output:** Table displayed in notebook and exported as `plots/monthly_price_table.png` for reports.

In [ ]:
# Pivot tables per year: total km and fuel by month (Jan–Dec order)
month_price_pivot_by_year = (
    fuel_data.pivot_table(
        index="MonthName",
        columns="Year",
        values={"Price": "sum"},
        aggfunc="sum",
    )
    .reindex(all_months["MonthName"])
    # .fillna(0)
)
month_price_pivot_by_year.loc["Mean [EUR]"] = month_price_pivot_by_year.astype(
    float).mean(axis=0).values
month_price_pivot_by_year["Mean [EUR]"] = month_price_pivot_by_year.astype(
    float).mean(axis=1).values
month_price_pivot_by_year = month_price_pivot_by_year.round(2)
dfi.export(month_price_pivot_by_year, 'plots/monthly_price_table.png', table_conversion='matplotlib', crop_top=False)
month_price_pivot_by_year